# Part 3 : Clustering

In [2]:
# The code was removed by Watson Studio for sharing.

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [4]:
!conda install -c conda-forge geopy --yes

!conda install -c conda-forge folium=0.5.0 --yes

print('Libraries installed!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.19.0               |             py_0          53 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.19.0-py_0       conda-forge

The following packages will be UPDATED:

   

In [5]:
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from pandas.io.json import json_normalize
import json

import requests

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported!')

Libraries imported!


In [6]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent = "tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical co-ordinates of Toronto are {} {} ".format(latitude , longitude))


The geographical co-ordinates of Toronto are 43.653963 -79.387207 


**Using Folium to create map of Toronto with appropriate markers of the Boroughs**  

In [7]:
map_toronto = folium.Map(location = [latitude, longitude] , zoom_start = 11)

for lat , long ,post ,borough,neigh in zip(df_final['Latitude'],df_final['Longitude'],df_final['Postcode'],df_final['Borough'],df_final['Neighbourhood']):
    label = "{} ({}) : {}".format(borough , post,neigh)
    popup = folium.Popup(label , parse_html = True)
    folium.CircleMarker([lat, long] , radius = 5 , popup = popup , color = 'blue' , fill = True , fill_color = '#3186cc' , fill_opacity = 0.7 , parse_html = False).add_to(map_toronto)
map_toronto    

**Exploring Boroughs with only Toronto in their names**

In [8]:
boroughs1 = ['East Toronto' , 'Central Toronto' , 'Downtown Toronto' , 'West Toronto']
df_final_new = df_final[df_final['Borough'].isin(boroughs1)].reset_index(drop = True)
print(df_final_new.shape)
df_final_new.head()

(38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [9]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, long, post, borough, neigh in zip(df_final_new['Latitude'], df_final_new['Longitude'], df_final_new['Postcode'], df_final_new['Borough'], df_final_new['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

**Using Foursquare API to explore the Boroughs**

In [34]:
# The code was removed by Watson Studio for sharing.

In [19]:
radius = 500 
LIMIT = 100
venues = []

for lat,long,post,borough,neighbourhood in zip(df_final_new['Latitude'] , df_final_new['Longitude'] , df_final_new['Postcode'] , df_final_new['Borough'] , df_final_new['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,lat,long,radius,LIMIT)
    
    results = requests.get(url).json()['response']['groups'][0]['items']

    for venue in results:
        venues.append((post,borough,neighbourhood,lat,long,venue['venue']['name'],venue['venue']['location']['lat'],venue['venue']['location']['lng'],venue['venue']['categories'][0]['name']))
        

In [21]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Postcode' , 'Borough' , 'Neighbourhood' , 'BoroughLatitude' , 'BoroughLongitude' , 'VenueName' , 'VenueLatitude' , 'VenueLongitude' , 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1701, 9)


,Postcode,Borough,Neighbourhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [22]:
venues_df.groupby(['Postcode' , 'Borough' , 'Neighbourhood'])['VenueName'].count()

Postcode  Borough           Neighbourhood                                                                                       
M4E       East Toronto      The Beaches                                                                                               4
M4K       East Toronto      The Danforth West,Riverdale                                                                              44
M4L       East Toronto      The Beaches West,India Bazaar                                                                            22
M4M       East Toronto      Studio District                                                                                          37
M4N       Central Toronto   Lawrence Park                                                                                             4
M4P       Central Toronto   Davisville North                                                                                          7
M4R       Central Toronto   North Toronto West         

**Number of venue categories in dataset**

In [23]:
len(venues_df['VenueCategory'].unique())


236

**ONE HOT ENCODING DIFFERENT CATEGORIES**

In [24]:
venues_onehot = pd.get_dummies(venues_df[['VenueCategory']] , prefix = "" , prefix_sep = "")

venues_onehot['Postcode'] = venues_df['Postcode']
venues_onehot['Borough'] = venues_df['Borough']
venues_onehot['Neighbourhoods'] = venues_df['Neighbourhood']

column1 = list(venues_onehot.columns[-3:])+ list(venues_onehot.columns[:-3])
venues_onehot = venues_onehot[column1]

print(venues_onehot.shape)
venues_onehot.head()


(1701, 239)


,Postcode,Borough,Neighbourhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Harbourfront,Regent Pa

**Calculating frequency of each category in neighbourhood**

In [25]:
df_freq = venues_onehot.groupby(['Postcode' , 'Borough' , 'Neighbourhoods']).mean().reset_index()
print(df_freq.shape)
df_freq.head()

(38, 239)


,Postcode,Borough,Neighbourhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000

**Top 10 occurences of venue types in a neighbourhood**

In [28]:
num = 10
indicators = ['st' , 'nd' , 'rd']

area_col = ['Postcode' , 'Borough' , 'Neighbourhoods']
freq_col = []
for gh in np.arange(num):
    try:
        freq_col.append('{}{} Most Common Venue'.format(gh+1 , indicators[ind]))
    except:
        freq_col.append('{}th Most Common Venue'.format(gh+1))
final_col = area_col + freq_col

df_sorted = pd.DataFrame(columns = final_col)
df_sorted['Postcode'] = df_freq['Postcode']
df_sorted['Borough'] = df_freq['Borough']
df_sorted['Neighbourhoods'] = df_freq['Neighbourhoods']

for gh in np.arange(df_freq.shape[0]):
    row_categories = df_freq.iloc[gh,:].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    df_sorted.iloc[gh , 3:] = row_categories_sorted.index.values[0:num]
    
df_sorted.sort_values(freq_col , inplace = True)
df_sorted

,Postcode,Borough,Neighbourhoods,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Airport Lounge,Plane,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate,Airport Food Court
32,M6J,West Toronto,"Little Portugal,Trinity",Bar,Coffee Shop,Asian Restaurant,New American Restaurant,Café,Cocktail Bar,Men's Store,Pizza Place,Bakery,Vietnamese Restaurant
33,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Grocery Store,Climbing Gym,Burrito Place,Stadium,Bar,Restaurant,Caribbean Restaurant
35,M6R,West Toronto,"Parkdale,Roncesvalles",Breakfast Spot,Gift Shop,Italian Restaurant,Dog Run,Movie Theater,Restaurant,Bookstore,Bank,Dessert Shop,Bar
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Cheese Shop,Fish Market,Bookstore,Coworking Space
34,M6P,West Toronto,"High Park,The Junction South",Café,Mexican Restaurant,Bar,Grocery Store,Arts & Crafts Store,Music Venue,Diner,Cajun / Creole Restaurant,Fast Food Restaurant,Bookstore
36,M6S,West Toronto,"Runnymede,Swansea",Café,Pizza Place,Coffee Shop,Sushi Restaurant,Gym,Italian Restaurant,Food & Drink Shop,Food,Bar,Dessert Shop
25,M5S,Downtown Toronto,"Harbord,University of Toronto",Café,Restaurant,Bakery,Japanese Restaurant,Bar,Bookstore,Sandwich Place,Beer Bar,Beer Store,Nightclub
26,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Coffee Shop,Mexican Restaurant,Bakery,Dumpling Restaurant,Vietnamese Restaurant,Chinese Restaurant,Cocktail Bar
6,M4R,Central Toronto,North Toronto West,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Fast Food Restaurant,Pet Store,Chinese Restaurant,Rental Car Location,Dessert Shop,Salon / Barbershop


**K Means Clustering**

In [32]:
k = 3

df_clustering = df_freq.drop(['Postcode' , 'Borough' , 'Neighbourhoods'] ,1)

kmeans = KMeans(n_clusters = k , random_state=0).fit(df_clustering)

df_clustered_df = df_final_new
df_clustered_df['Cluster'] = kmeans.labels_

df_clustered_df = df_clustered_df.join(df_sorted.drop(['Borough' , 'Neighbourhoods'],1).set_index('Postcode') , on='Postcode')
df_clustered_df.sort_values(['Cluster'] + freq_col, inplace=True)
df_clustered_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M6J,West Toronto,"Little Portugal,Trinity",43.647927,-79.419750,0,Bar,Coffee Shop,Asian Restaurant,New American Restaurant,Café,Cocktail Bar,Men's Store,Pizza Place,Bakery,Vietnamese Restaurant
8,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259,0,Pharmacy,Supermarket,Bakery,Gym / Fitness Center,Fast Food Restaurant,Liquor Store,Middle Eastern Restaurant,Discount Store,Music Venue,Park
31,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420,1,Airport Service,Airport Terminal,Airport Lounge,Plane,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate,Airport Food Court
13,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",43.636847,-79.428191,1,Breakfast Spot,Café,Coffee Shop,Grocery Store,Climbing Gym,Burrito Place,Stadium,Bar,Restaurant,Caribbean Restaurant
24,M6R,West Toronto,"Parkdale,Roncesvalles",43.648960,-79.456325,1,Breakfast Spot,Gift Shop,Italian Restaurant,Dog Run,Movie Theater,Restaurant,Bookstore,Bank,Dessert Shop,Bar
16,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Cheese Shop,Fish Market,Bookstore,Coworking Space
21,M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763,1,Café,Mexican Restaurant,Bar,Grocery Store,Arts & Crafts Store,Music Venue,Diner,Cajun / Creole Restaurant,Fast Food Restaurant,Bookstore
27,M6S,West Toronto,"Runnymede,Swansea",43.651571,-79.484450,1,Café,Pizza Place,Coffee Shop,Sushi Restaurant,Gym,Italian Restaurant,Food & Drink Shop,Food,Bar,Dessert Shop
26,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049,1,Café,Restaurant,Bakery,Japanese Restaurant,Bar,Bookstore,Sandwich Place,Beer Bar,Beer Store,Nightclub
29,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",43.653206,-79.400049,1,Café,Vegetarian / Vegan Restaurant,Bar,Coffee Shop,Mexican Restaurant,Bakery,Dumpling Restaurant,Vietnamese Restaurant,Chinese Restaurant,Cocktail Bar


**Ilustrating these clusters on the map**

In [33]:
map_clusters = folium.Map(location=[latitude,longitude] ,zoom_start=12 )

x = np.arange(k)
ys = [i+x+(i*x)**2 for i  in range(k)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat,lon,post,bor,poi,cluster in zip(df_clustered_df['Latitude'] , df_clustered_df['Longitude'] , df_clustered_df['Postcode'] , df_clustered_df['Borough'] , df_clustered_df['Neighbourhood'] , df_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters